In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
from dataset.get_dataset import getDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import glob

### List Checkpoints

In [102]:
root = '../outputs'
checkpoints = sorted(glob.glob(f'{root}/**/best_loss.pt', recursive=True))
len(checkpoints)

8

In [103]:
checkpoints = [chk for chk in checkpoints if 'majority' not in chk and 'intersection' not in chk]

In [104]:
checkpoints

['../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
 '../outputs/RIGA_gt_annots_train_A2_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
 '../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
 '../outputs/RIGA_gt_annots_train_A4_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
 '../outputs/RIGA_gt_annots_train_A5_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
 '../outputs/RIGA_gt_annots_train_A6_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt']

### DataLoader

In [6]:
class args_struct:
    def __init__(self):
        self.dataroot = '../../data/DiscRegion/DiscRegion/'
        self.rater_num = 6
        self.img_width = 256
        self.img_height = 256
        self.dataset = 'RIGA'
        self.validate = True
        self.standardize = False
        self.random_seed = 27
        self.loss_func = 'bce'
        self.gt_type_train = -1

In [7]:
args = args_struct()
train_set, valid_set, test_set = getDataset(args, validate=args.validate)

In [8]:
len(valid_set)

131

In [9]:
val_loader = DataLoader(valid_set, batch_size=16, shuffle=True, num_workers=4)

### Val Function

In [10]:
from trainer.train_riga import validate_riga_tab
from loss_func.get_loss import get_loss_func

In [20]:
args.gt_type_train = -1
args.loss_func = 'bce' 
args.batch_size = 8
args.num_worker = 4
args.use_mix_label = False
loss_func = get_loss_func(args)

In [12]:
from models.unet import UNet
model = UNet()

In [105]:
from collections import defaultdict

In [151]:
checkpoint_to_metrics_val = defaultdict(dict)

for chkp in checkpoints:
    print(chkp)
    params = torch.load(chkp)
    model_params = params['model']
    model.load_state_dict(model_params)
    
    loss, dice_disc, dice_cup = validate_riga_tab(args, model, valid_set, loss_func)
    checkpoint_to_metrics_val[chkp]['loss'] = round(loss, 4)
    checkpoint_to_metrics_val[chkp]['dice_disc'] = round(dice_disc, 4)
    checkpoint_to_metrics_val[chkp]['dice_cup'] = round(dice_cup, 4)

../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


100%|██████████| 17/17 [00:02<00:00,  6.50it/s]


../outputs/RIGA_gt_annots_train_A2_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


100%|██████████| 17/17 [00:02<00:00,  6.85it/s]


../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


...

100%|██████████| 17/17 [00:02<00:00,  6.60it/s]


../outputs/RIGA_gt_annots_train_A5_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


100%|██████████| 17/17 [00:02<00:00,  6.68it/s]


../outputs/RIGA_gt_annots_train_A6_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


100%|██████████| 17/17 [00:02<00:00,  6.70it/s]


In [152]:
checkpoint_to_metrics_val

defaultdict(dict,
            {'../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt': {'loss': 0.0887,
              'dice_disc': 0.9553,
              'dice_cup': 0.766},
             '../outputs/RIGA_gt_annots_train_A2_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt': {'loss': 0.1168,
              'dice_disc': 0.893,
              'dice_cup': 0.7916},
             '../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt': {'loss': 0.0919,
              'dice_disc': 0.9582,
              'dice_cup': 0.721},
             '../outputs/RIGA_gt_annots_train_A4_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt': {'loss': 0.0952,
              'dice_disc': 0.9599,
              'dice_cup': 0.7938},
             '../outputs/RIGA_gt_annots_train_A5_Unet_bce_pretrain0_v

In [121]:
def do_swa(checkpoint):
	skip = ['relative_position_index', 'num_batches_tracked', 'decoder', 'logit']

	K = len(checkpoint)
	swa = None
	
	for k in range(K):
		state_dict = torch.load(checkpoint[k], map_location=lambda storage, loc: storage)['model']
		if swa is None:
			swa = state_dict
		else:
			for k, v in state_dict.items():
				# print(k)
				if any(s in k for s in skip): continue
				swa[k] += v
	
	for k, v in swa.items():
		if any(s in k for s in skip): continue
		swa[k] /= K
	
	return swa

### Uniform Soup

In [123]:
swa = do_swa(checkpoints)
model.load_state_dict(swa)
pritn('Decoder, logit skipped')
validate_riga_tab(args, model, valid_set, loss_func)

100%|██████████| 17/17 [00:02<00:00,  6.87it/s]


(0.14615696907498454, 0.8884378339498098, 0.35799460374671993)

In [124]:
validate_riga_tab(args, model, test_set, loss_func)

100%|██████████| 12/12 [00:04<00:00,  2.99it/s]


(0.1517971783876419, 0.8970697722936931, 0.28713531839220147)

### Greedy Soup

In [147]:
def get_mean_dice(dice_cup, dice_disc):
    return (dice_cup + dice_disc) / 2

def sort_dict(x):
    return [(k, v) for k, v in sorted(x.items(), key=lambda item: get_mean_dice(item[1]['dice_cup'], item[1]['dice_disc']), reverse=True)]

In [153]:
checkpoint_to_metrics_val_sorted = sort_dict(checkpoint_to_metrics_val)

In [154]:
checkpoint_to_metrics_val_sorted

[('../outputs/RIGA_gt_annots_train_A4_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
  {'loss': 0.0952, 'dice_disc': 0.9599, 'dice_cup': 0.7938}),
 ('../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
  {'loss': 0.0887, 'dice_disc': 0.9553, 'dice_cup': 0.766}),
 ('../outputs/RIGA_gt_annots_train_A2_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
  {'loss': 0.1168, 'dice_disc': 0.893, 'dice_cup': 0.7916}),
 ('../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
  {'loss': 0.0919, 'dice_disc': 0.9582, 'dice_cup': 0.721}),
 ('../outputs/RIGA_gt_annots_train_A6_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt',
  {'loss': 0.1119, 'dice_disc': 0.9507, 'dice_cup': 0.6339}),
 ('../outputs/RIGA_gt_annots_train_A5_

In [161]:
best_val_dice = get_mean_dice(
    checkpoint_to_metrics_val_sorted[0][1]['dice_disc'],
    checkpoint_to_metrics_val_sorted[0][1]['dice_cup']
)

greedy_chks = []
print('Decoder, logit skipped')
print('best_val_dice', best_val_dice)
for chk in checkpoint_to_metrics_val_sorted:
    chk = chk[0]
    swa = do_swa(greedy_chks + [chk])
    model.load_state_dict(swa)
    loss, dice_disc, dice_cup = validate_riga_tab(args, model, valid_set, loss_func)
    print(chk) 
    print(loss, dice_disc, dice_cup)
    print(get_mean_dice(dice_disc, dice_cup))

    if get_mean_dice(dice_disc, dice_cup) >= best_val_dice:
        best_val_dice = get_mean_dice(dice_disc, dice_cup)
        greedy_chks.append(chk)

Decoder, logit skipped
best_val_dice 0.8768499999999999


100%|██████████| 17/17 [00:02<00:00,  6.53it/s]


../outputs/RIGA_gt_annots_train_A4_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
0.09523010242531318 0.9599325675090761 0.7937565100102024
0.8768445387596393


100%|██████████| 17/17 [00:02<00:00,  6.83it/s]


../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
0.08872173356873389 0.9553255984801372 0.7660221648580245
0.8606738816690809


...

../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
0.09186874301606462 0.9581757997738496 0.720991621035656
0.8395837104047528


100%|██████████| 17/17 [00:02<00:00,  6.71it/s]


../outputs/RIGA_gt_annots_train_A6_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
0.1119334957071843 0.9506568066946423 0.6338628289353756
0.7922598178150089


100%|██████████| 17/17 [00:02<00:00,  6.61it/s]

../outputs/RIGA_gt_annots_train_A5_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
0.17660424556895976 0.8329441642943229 0.4623990024774129
0.6476715833858679


In [162]:
greedy_chks

[]

In [118]:
uniform_soup_to_metrics_val = defaultdict(dict)
NUM_MODELS = len(checkpoints)

for i, chkp in enumerate(checkpoints):
    print(chkp)
    state_dict = torch.load(chkp)['model']
    if i == 0:
        uniform_soup = {k : v * (1./NUM_MODELS) for k, v in state_dict.items()}
    else:
        uniform_soup = {k : v * (1./NUM_MODELS) + uniform_soup[k] for k, v in state_dict.items()}

model.load_state_dict(uniform_soup)
loss, dice_disc, dice_cup = validate_riga_tab(args, model, valid_set, loss_func)
print(loss, dice_disc, dice_cup)

../outputs/RIGA_gt_annots_train_A1_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
../outputs/RIGA_gt_annots_train_A2_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
../outputs/RIGA_gt_annots_train_A3_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
../outputs/RIGA_gt_annots_train_A4_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
../outputs/RIGA_gt_annots_train_A5_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt
../outputs/RIGA_gt_annots_train_A6_Unet_bce_pretrain0_validateTrue/bs16_lr5e-05_wd0.0_epoch200/loop0/checkpoints/best_loss.pt


100%|██████████| 17/17 [00:02<00:00,  6.55it/s]

0.46176526746677077 0.14144438425548203 0.025471888752725286


In [117]:
model.load_state_dict(swa)
validate_riga_tab(args, model, valid_set, loss_func)

100%|██████████| 17/17 [00:02<00:00,  6.36it/s]


(0.09218285204345034, 0.9532810567899515, 0.8039528763931216)

In [78]:
print(loss, dice_disc, dice_cup)

0.47844331319095523 0.14144438425548203 0.025471888752725286


In [82]:
state_dict.keys()

odict_keys(['encoder.conv1.weight', 'encoder.bn1.weight', 'encoder.bn1.bias', 'encoder.bn1.running_mean', 'encoder.bn1.running_var', 'encoder.bn1.num_batches_tracked', 'encoder.layer1.0.conv1.weight', 'encoder.layer1.0.bn1.weight', 'encoder.layer1.0.bn1.bias', 'encoder.layer1.0.bn1.running_mean', 'encoder.layer1.0.bn1.running_var', 'encoder.layer1.0.bn1.num_batches_tracked', 'encoder.layer1.0.conv2.weight', 'encoder.layer1.0.bn2.weight', 'encoder.layer1.0.bn2.bias', 'encoder.layer1.0.bn2.running_mean', 'encoder.layer1.0.bn2.running_var', 'encoder.layer1.0.bn2.num_batches_tracked', 'encoder.layer1.0.conv3.weight', 'encoder.layer1.0.bn3.weight', 'encoder.layer1.0.bn3.bias', 'encoder.layer1.0.bn3.running_mean', 'encoder.layer1.0.bn3.running_var', 'encoder.layer1.0.bn3.num_batches_tracked', 'encoder.layer1.0.downsample.0.weight', 'encoder.layer1.0.downsample.1.weight', 'encoder.layer1.0.downsample.1.bias', 'encoder.layer1.0.downsample.1.running_mean', 'encoder.layer1.0.downsample.1.running